# 機械学習帳 確認問題(6.11)

[機械学習帳の線形二値分類の確認問題(6.11)](https://chokkan.github.io/mlnote/classification/02multi.html#id16)を解きます。

## 講義内容の復習

![chapter6-1.svg](./images/chapter6/chapter6-1.svg)

![chapter6-2.svg](./images/chapter6/chapter6-2.svg)

![chapter6-3.svg](./images/chapter6/chapter6-3.svg)

## 0. データ準備

確認問題に使用するMNISTのデータを準備します。

In [1]:
import gzip
import sys
import struct
import urllib.request
from pathlib import Path
import numpy as np

def read_image(fi):
    magic, n, rows, columns = struct.unpack(">IIII", fi.read(16))
    assert magic == 0x00000803
    assert rows == 28
    assert columns == 28
    rawbuffer = fi.read()
    assert len(rawbuffer) == n * rows * columns
    rawdata = np.frombuffer(rawbuffer, dtype='>u1', count=n*rows*columns)
    return rawdata.reshape(n, rows, columns).astype(np.float32) / 255.0

def read_label(fi):
    magic, n = struct.unpack(">II", fi.read(8))
    assert magic == 0x00000801
    rawbuffer = fi.read()
    assert len(rawbuffer) == n
    return np.frombuffer(rawbuffer, dtype='>u1', count=n)

def openurl_gzip(url):
    request = urllib.request.Request(
        url,
        headers={
            "Accept-Encoding": "gzip",
            "User-Agent": "Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11", 
        })
    response = urllib.request.urlopen(request)
    return gzip.GzipFile(fileobj=response, mode='rb')

def save_mnist():
    if Path("data/mnist.npz").exists():
        return
    np.savez_compressed(
        "data/mnist",
        train_x=read_image(openurl_gzip("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")),
        train_y=read_label(openurl_gzip("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")),
        test_x=read_image(openurl_gzip("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")),
        test_y=read_label(openurl_gzip("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"))
    )

save_mnist()
data = np.load("data/mnist.npz")
print("Training data (X):", data["train_x"].shape, data["train_x"].dtype)
print("Training data (Y):", data["train_y"].shape, data["train_y"].dtype)
print("Test data (X):", data["test_x"].shape, data["test_x"].dtype)
print("Test data (Y):", data["test_y"].shape, data["test_y"].dtype)

Training data (X): (60000, 28, 28) float32
Training data (Y): (60000,) uint8
Test data (X): (10000, 28, 28) float32
Test data (Y): (10000,) uint8


## 1. 確率的勾配降下法による多クラスロジスティック回帰モデルの学習

確率的勾配降下法で多クラスロジスティック回帰モデルを学習するアルゴリズムを実装せよ。学習データと評価データはMNISTを用いよ。

In [2]:
class LogisticClassifier():
    
    def __init__(self):
        self.W = None

    def softmax(self, a):
        # refer 6.5.1
        ea = np.exp(a - np.max(a))
        return ea / np.sum(ea)

    def train(self, X, Y, num_class, eta=1e-3, alpha=1e-6, epoch=100000, eps=1e-6):
        N = X.shape[0]
        self.W = np.random.uniform(size=(X.shape[-1], num_class))
        for t in range(epoch):
            i = np.random.choice(N)
            hat_y = self.predict_proba(X[i])
            # to one-hot vector
            y = np.zeros(num_class)
            y[Y[i]] = 1.0
            delta = (y - hat_y) * X[i].reshape((-1, 1)) - 2 * alpha * self.W / N
            if np.sum(np.abs(delta)) < eps:
                break
            self.W += eta * delta
        return self

    def predict_proba(self, x):
        value = x @ self.W
        if len(value.shape) < 2:
            return self.softmax(value).flatten()
        else:
            return np.apply_along_axis(self.softmax, axis=1, arr=value)

    def predict(self, x):
        proba = self.predict_proba(x)
        return np.argmax(proba, axis=1)

softmax関数が実装できているかテストします。

In [3]:
# refer 6.9
np.testing.assert_almost_equal(
    1.0,
    np.sum(LogisticClassifier().softmax(np.array([3, 2, -5])))
)

# refer 6.10
assert np.all((LogisticClassifier().softmax(np.array([3, 2, -5])) > 0) == True)

# refer 6.11
np.testing.assert_almost_equal(
    1.0 / (1 + np.exp(-(0.5 - 0.1))),
    LogisticClassifier().softmax(np.array([0.5, 0.1]))[0]
)

学習を行います。

In [4]:
X_train = data["train_x"]


def to_feature(X):
    return np.c_[X.reshape((X.shape[0], -1)), np.ones(X.shape[0])]


X_train = to_feature(X_train)
model = LogisticClassifier().train(X=X_train, Y=data["train_y"], num_class=10)

## 2. 評価データ上での正解率

評価データ上で学習したモデルの正解率を測定せよ。

In [5]:
X_test = data["test_x"]
X_test = to_feature(X_test)

In [6]:
from sklearn.metrics import classification_report


predictions = model.predict(X_test)
print(classification_report(data["test_y"], predictions))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95       980
           1       0.95      0.96      0.96      1135
           2       0.89      0.87      0.88      1032
           3       0.88      0.86      0.87      1010
           4       0.89      0.90      0.90       982
           5       0.83      0.82      0.82       892
           6       0.92      0.92      0.92       958
           7       0.88      0.90      0.89      1028
           8       0.84      0.82      0.83       974
           9       0.86      0.85      0.86      1009

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



正解率であるaccuracyは0.89となります。